In [2]:
out_dir = "./battle_013_vs_017"

In [1]:
# https://github.com/Lgeu/hungry_geese/commit/465f300c94c2c4eb2912e1381bc9fb23c9303512
!g++ ../../../KKT89/Combine.cpp -O3 -Ofast -march=native -std=c++17

In file included from ../../../KKT89/src/Goose.hpp:3,
                 from ../../../KKT89/src/Stage.hpp:4,
                 from ../../../KKT89/src/Duct.hpp:7,
                 from ../../../KKT89/src/Duct.cpp:2,
                 from ../../../KKT89/Combine.cpp:3:
../../../KKT89/src/Evaluator.hpp: In member function ‘void hungry_geese::evaluation_function::BitBoard::ShiftRight()’:
../../../KKT89/src/library.hpp:64:9: warning: format ‘%d’ expects argument of type ‘int’, but argument 3 has type ‘hungry_geese::evaluation_function::BitBoard::ull’ {aka ‘long long unsigned int’} [-Wformat=]
   64 |         "`%s` (%d) is out of range [%d, %d)", #value, value, left, right)
      |         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../../KKT89/src/library.hpp:54:24: note: in definition of macro ‘ASSERT’
   54 |                 printf(__VA_ARGS__); \
      |                        ^~~~~~~~~~~
../../../KKT89/src/Evaluator.hpp:664:9: note: in expansion of macro ‘ASSERT_RANGE’
  664 |         ASSERT_

In file included from ../../../KKT89/src/Duct.hpp:10,
                 from ../../../KKT89/src/Duct.cpp:2,
                 from ../../../KKT89/Combine.cpp:3:
../../../KKT89/src/Evaluator.hpp: In member function ‘void hungry_geese::evaluation_function::Model<in_dim, out_dim, hidden_1, hidden_2>::LoadParameters(const char*) [with int in_dim = 2382; int out_dim = 5; int hidden_1 = 256; int hidden_2 = 32]’:
../../../KKT89/src/Evaluator.hpp:492:14: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  492 |         fread(&embed.parameters.weight, embed.parameters.weight.Ravel().size(), sizeof(short), fp);
      |         ~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
../../../KKT89/src/Evaluator.hpp:493:14: warning: ignoring return value of ‘size_t fread(void*, size_t, size_t, FILE*)’, declared with attribute warn_unused_result [-Wunused-result]
  493 |      

In [2]:
!mkdir {out_dir}
!mkdir {out_dir}/1vs3
!mkdir {out_dir}/2vs2
!mkdir {out_dir}/3vs1

In [3]:
# コマンド確認
param1 = "../../rl/parameters/013_01.bin"
param2 = "../../rl/parameters/017_02.bin"
!./a.out -j -o ./ -t 0.1 -p {param1} {param2} {param1} {param2}

In [4]:
from subprocess import Popen
N_MATCHES = 50000
p1 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o {out_dir}/1vs3/ -t 0.1 -p {param1} {param2} {param2} {param2}'", shell=True)
p2 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o {out_dir}/2vs2/ -t 0.1 -p {param1} {param1} {param2} {param2}'", shell=True)
p3 = Popen(f"seq 1 {N_MATCHES} | xargs -I@ -P 20 sh -c './a.out -j -o {out_dir}/3vs1/ -t 0.1 -p {param1} {param1} {param1} {param2}'", shell=True)

In [3]:
!ls -1 {out_dir}/1vs3 | wc -l
!ls -1 {out_dir}/2vs2 | wc -l
!ls -1 {out_dir}/3vs1 | wc -l

4179
4221
4002


In [4]:
import numpy as np

ranks_param2 = []
files = !ls -1 {out_dir}/3vs1
for file in files:
    ranks = !tail {out_dir}/3vs1/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[-1])
rank_count = np.bincount(ranks_param2)[1:]
print("3vs1", rank_count, rank_count / rank_count.sum())

ranks_param2 = []
files = !ls -1 {out_dir}/2vs2
for file in files:
    ranks = !tail {out_dir}/2vs2/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[2])
    ranks_param2.append(ranks[3])
rank_count = np.bincount(ranks_param2)[1:]
print("2vs2", rank_count, rank_count / rank_count.sum())

ranks_param2 = []
files = !ls -1 {out_dir}/1vs3
for file in files:
    ranks = !tail {out_dir}/1vs3/{file} -n 1
    ranks = list(map(int, ranks[0].split()))
    ranks_param2.append(ranks[1])
    ranks_param2.append(ranks[2])
    ranks_param2.append(ranks[3])
rank_count = np.bincount(ranks_param2)[1:]
print("1vs3", rank_count, rank_count / rank_count.sum())

3vs1 [1750  608  761  883] [0.43728136 0.15192404 0.19015492 0.22063968]
2vs2 [3509 1818 1507 1608] [0.4156598  0.21535181 0.1785122  0.19047619]
1vs3 [4442 3168 2574 2353] [0.35431124 0.25269203 0.20531228 0.18768445]


In [5]:
!zip -r -q 013_vs_017.zip {out_dir}